In [21]:
from tkinter import Tk
from tkinter import Label
from tkinter import Entry
from tkinter import Button
from tkinter import Frame
from tkinter import END
from os.path import join
from time import sleep
from tkinter.messagebox import showerror

from datetime import datetime
from win32com.client import GetObject
from win32com.client import CDispatch
from subprocess import Popen
from subprocess import call
from time import sleep
from ast import literal_eval

from pyautogui import press
from pyautogui import click

from cryptography.fernet import Fernet

with open(b"bin/v.key", "rb") as keygen:
    key = keygen.readline()
fernet = Fernet(key)
#print(key)
with open(b"bin/v.lnc", "rb") as file:
    encdata = file.readline()
#print(encdata)
decoded_data = fernet.decrypt(encdata).decode()
validation_date =datetime.strptime(decoded_data, "%Y-%m-%d")
    #print(validation_date)




monitoring_time_delay = 3


class SapLogin():
    """
    DESCRIPTION
    SAP automation module for python
    ================================
    Saplogin is a module which will help to automate the SAP related manual workload.
    
    
    USE:
        1. create a object of SapLogin class with correct argument-
            $sapObject = SapLogin(sap_id, sap_pw, sap_connection)
        2. After creating the object, user need to login.-
            $sapObject.login()
        3. After login we can set the flowchart of multiple activities-
            for example we want to confirm '05 Stage':
            $sapObject.open_zscan() ------> open the zscan window,
            $sapObject.stage_confirmation_05(module_id="50984324") ------>confirmation in 05 stage window.
            $sapObject.back(1) -------> back to it's previous window
    """
    
    def __init__(self, sap_id, sap_pw, sap_connection):
        
        self.sap_id = sap_id
        self.sap_pw = sap_pw
        self.sap_connection = sap_connection
        self._sapPath_ = r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe" 
        self._timeDelay_ = 1
        
    def login(self):
        """With this function we can login in SAP"""
        Popen(self._sapPath_)
        sleep(5)
        SapGuiAuto = GetObject("SAPGUI")
        
        if not type(SapGuiAuto) == CDispatch:
            return
        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == CDispatch:
            self.SapGuiAuto = None
            return
        
        connection = application.OpenConnection(self.sap_connection, True)
        
        if not type(connection) == CDispatch:
            self.SapGuiAuto = None
            self.application = None
            return
        
        self.session = connection.Children(0)
        
        if not type(self.session) == CDispatch:
            self.SapGuiAuto = None
            self.application = None
            #self.session = None
            return
        self.session.findById("wnd[0]/usr/txtRSYST-BNAME").text = self.sap_id
        self.session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = self.sap_pw
        self.session.findById("wnd[0]").sendVKey(0)
        
    def open_zscan(self):
        """using this function we can open zscan window"""
        self.session.findById("wnd[0]/tbar[0]/okcd").text = "zscan"
        self.session.findById("wnd[0]").sendVKey(0)
        #self.session.findById("wnd[0]").sendVKey(8)
        
        
    def stage_confirmation_05(self, module_id):
        """After opening the zscan window """
        self.open_zscan()
        self.session.findById("wnd[0]").sendVKey(8)
        self.session.findById("wnd[0]/usr/subSUB2:ZBARCODE_SCANNER_COPY:0902/txtP_SN").text = module_id
        self.session.findById("wnd[0]/usr/subSUB2:ZBARCODE_SCANNER_COPY:0902/txtP_SN").caretPosition = 6
        self.session.findById("wnd[0]").sendVKey(0)      
        self.session.findById("wnd[0]").sendVKey(11)
        self.session.findById("wnd[0]").sendVKey(0)
        self.back(2)
        
    def stage_confirmation_10(self, module_id):
        self.open_zscan()
        call("cmd /c stage_10_selection.vbs")
        self.session.findById("wnd[0]/usr/subSUB3:ZBARCODE_SCANNER_COPY:0903/txtP_SN1").text = module_id
        self.session.findById("wnd[0]/usr/subSUB3:ZBARCODE_SCANNER_COPY:0903/txtP_SN1").caretPosition = 6
        self.session.findById("wnd[0]").sendVKey(0)
        self.session.findById("wnd[0]").sendVKey(11)
        self.back(2)

    def back(self, number_of_time = 1):
        for _ in range(0, number_of_time):
            self.session.findById("wnd[0]").sendVKey(3)
def read_config(config_path):
    with open(config_path, "r") as file:
        data = file.read()
    return literal_eval(data)

config_path = "config.txt"
config_data = read_config(config_path)
sap_id = config_data["sap_id"]
sap_pw = config_data["sap_pw"]
data_saving_path = config_data["data_file_path"]
sap_connection  = config_data["sap_connection"]
all_el_line = config_data["all_el_line"]
_current_el_line = config_data["current_el_line"] 
timedelay = config_data["timedelay"]

sapSession = SapLogin(sap_id, sap_pw, sap_connection)
sapSession.login()

def submit_confirmation(event=None):
    present_time = datetime.now()
    print(validation_date)
    if validation_date.year >= present_time.year and validation_date.month >= present_time.month and validation_date.day >= present_time.day:

        module_id = module_input_entry.get()

        timestamp = present_time.strftime("%y/%m/%d %H%M%S")
        sapSession.stage_confirmation_05(module_id)
        sleep(timedelay)
        sapSession.stage_confirmation_10(module_id)

        data_log = "{},{},{}".format(module_id, _current_el_line, timestamp)
        data_for_label = "Module Id-{} Line-{} Timestamp-{}".format(module_id, 
                                                                    _current_el_line,
                                                                    timestamp)

        data_saving_file = join(data_saving_path, 
                                _current_el_line, 
                                present_time.strftime("%y%m%d.txt"))
        with open(data_saving_file, "a", newline="") as file:
            file.write(data_log+"\n")

        print(data_log)
        datalog_label = Label(frame_datashow, text = data_for_label)
        datalog_label.grid(row=0, column=0)

        module_input_entry.delete(0,END)
    
    else:
        showerror("Error", "Error Code 101, Validation Related issue. Please contact with Deep Gupta(Phone No: +91-9153436296).!!!!")
        

    
root = Tk()
icon_img = r"img/logo.ico"
root.title("10-Stage Confirmation")
root.iconbitmap(icon_img)

frame_input = Frame(root)
frame_input.grid(row=0, column=0)

frame_datashow = Frame(root)
frame_datashow.grid(row=1, column=0)

module_input_label = Label(frame_input, text="Scan Module ID:", borderwidth=2, relief="groove")
module_input_label.grid(row=0, column=0)


module_input_entry = Entry(frame_input)
module_input_entry.grid(row=0, column=1)

module_submit_btn = Button(frame_input, text = "Submit", command = submit_confirmation)
module_submit_btn.grid(row=0, column=2)

module_input_entry.bind('<Return>', submit_confirmation)

root.mainloop()

2021-09-28 00:00:00
1231233,EL1,21/09/28 115851
2021-09-28 00:00:00
1433234,EL1,21/09/28 115905


In [16]:
from cryptography.fernet import Fernet
#key = Fernet.generate_key()
#print(key)
#print(type(key))
date = "2021-09-27"
fernet = Fernet(b"tAqupJuLkpy1Vir3E0hyOy9ggquK4CMVrR4p3khAL25=")
encdate = fernet.encrypt(date.encode())
print(encdate)
with open("valid.key", "wb") as file:
    file.write(encdate)
with open("valid.key", "rb") as file:
    encdata = file.readline()
print(encdata)
decData = fernet.decrypt(encdata).decode()
print(decData)

b'gAAAAABhUbiWsfG8i1r5pIgbLMKOEMetmtMAv-MC-ce3bcgedC013bVColGsQHxaXQOQjT-DsuLWHjlJkcuQJMki_xgrH6IqkQ=='
b'gAAAAABhUbiWsfG8i1r5pIgbLMKOEMetmtMAv-MC-ce3bcgedC013bVColGsQHxaXQOQjT-DsuLWHjlJkcuQJMki_xgrH6IqkQ=='
2021-09-27


In [10]:
from tkinter import Tk
from tkinter import Label
from tkinter import Entry
from tkinter import Button
from cryptography.fernet import Fernet

root = Tk()

label_input = Label(root,text="Date:")
label_input.grid(row=0,column=0)

entry = Entry(root)
entry.grid(row=0, column=1)

def generate():
    key = Fernet.generate_key()
    fernet = Fernet(key)
            
    date_string  = entry.get()
    encmsg = fernet.encrypt(date_string.encode())
    
    with open("v.key", "wb") as keygen:
        keygen.write(key)
    print(key)
    with open("v.lnc", "wb") as file:
        file.write(encmsg)
        
    done_label = Label(root, text="Done!!;-)")
    done_label.grid(row=1,column=0)
        
btn  = Button(root, text = "Generate Key", command = generate)
btn.grid(row=0, column=2)

root.mainloop()

b'hQEiwQ_WZ75_h6RgmM9j3zG1hhGy_C0lh2qUJAsSmzk='


In [8]:
from cryptography.fernet import Fernet
#key = Fernet.generate_key()
with open("v.key", "rb") as keygen:
    key = keygen.readline()
fernet = Fernet(key)
print(key)
with open("v.lnc", "rb") as file:
    encdata = file.readline()
print(encdata)
decData = fernet.decrypt(encdata).decode()
print(decData)

b'T6EoTiHRMxaiviCerwV_nq4VLItdvVVVB8j9fIBcma4='
b'gAAAAABhUpo1cDVMD_EKkAplPyWrkSIOhnX2pe3GrJEkRGjdDffsfcs5z7ORdEPQqjLRQ_5A-w9CUnTpChnRAXbYdzDwBaUQSQ=='
what the hell


In [12]:
from tkinter import Tk
from tkinter import Label
from tkinter import Entry
from tkinter import Button
from tkinter import Frame
from tkinter import END
from os.path import join
from time import sleep
from tkinter.messagebox import showerror

from datetime import datetime
from win32com.client import GetObject
from win32com.client import CDispatch
from subprocess import Popen
from subprocess import call
from time import sleep
from ast import literal_eval

from pyautogui import press
from pyautogui import click

from cryptography.fernet import Fernet

with open(b"bin/v.key", "rb") as keygen:
    key = keygen.readline()
fernet = Fernet(key)
#print(key)
with open(b"bin/v.lnc", "rb") as file:
    encdata = file.readline()
#print(encdata)
decoded_data = fernet.decrypt(encdata).decode()
validation_date =datetime.strptime(decoded_data, "%Y-%m-%d")
    #print(validation_date)




monitoring_time_delay = 3


class SapLogin():
    """
    DESCRIPTION
    SAP automation module for python
    ================================
    Saplogin is a module which will help to automate the SAP related manual workload.
    
    
    USE:
        1. create a object of SapLogin class with correct argument-
            $sapObject = SapLogin(sap_id, sap_pw, sap_connection)
        2. After creating the object, user need to login.-
            $sapObject.login()
        3. After login we can set the flowchart of multiple activities-
            for example we want to confirm '05 Stage':
            $sapObject.open_zscan() ------> open the zscan window,
            $sapObject.stage_confirmation_05(module_id="50984324") ------>confirmation in 05 stage window.
            $sapObject.back(1) -------> back to it's previous window
    """
    
    def __init__(self, sap_id, sap_pw, sap_connection):
        
        self.sap_id = sap_id
        self.sap_pw = sap_pw
        self.sap_connection = sap_connection
        self._sapPath_ = r"C:\Program Files (x86)\SAP\FrontEnd\SAPgui\saplogon.exe" 
        self._timeDelay_ = 1
        
    def login(self):
        """With this function we can login in SAP"""
        Popen(self._sapPath_)
        sleep(5)
        SapGuiAuto = GetObject("SAPGUI")
        
        if not type(SapGuiAuto) == CDispatch:
            return
        application = SapGuiAuto.GetScriptingEngine
        if not type(application) == CDispatch:
            self.SapGuiAuto = None
            return
        
        connection = application.OpenConnection(self.sap_connection, True)
        
        if not type(connection) == CDispatch:
            self.SapGuiAuto = None
            self.application = None
            return
        
        self.session = connection.Children(0)
        
        if not type(self.session) == CDispatch:
            self.SapGuiAuto = None
            self.application = None
            #self.session = None
            return
        self.session.findById("wnd[0]/usr/txtRSYST-BNAME").text = self.sap_id
        self.session.findById("wnd[0]/usr/pwdRSYST-BCODE").text = self.sap_pw
        self.session.findById("wnd[0]").sendVKey(0)
        
    def open_zscan(self):
        """using this function we can open zscan window"""
        self.session.findById("wnd[0]/tbar[0]/okcd").text = "zscan"
        self.session.findById("wnd[0]").sendVKey(0)
        #self.session.findById("wnd[0]").sendVKey(8)
        
        
    def stage_confirmation_05(self, module_id):
        """After opening the zscan window """
        self.open_zscan()
        self.session.findById("wnd[0]").sendVKey(8)
        self.session.findById("wnd[0]/usr/subSUB2:ZBARCODE_SCANNER_COPY:0902/txtP_SN").text = module_id
        self.session.findById("wnd[0]/usr/subSUB2:ZBARCODE_SCANNER_COPY:0902/txtP_SN").caretPosition = 6
        self.session.findById("wnd[0]").sendVKey(0)      
        self.session.findById("wnd[0]").sendVKey(11)
        self.session.findById("wnd[0]").sendVKey(0)
        self.back(2)
        
    def stage_confirmation_10(self, module_id):
        self.open_zscan()
        call("cmd /c stage_10_selection.vbs")
        self.session.findById("wnd[0]/usr/subSUB3:ZBARCODE_SCANNER_COPY:0903/txtP_SN1").text = module_id
        self.session.findById("wnd[0]/usr/subSUB3:ZBARCODE_SCANNER_COPY:0903/txtP_SN1").caretPosition = 6
        self.session.findById("wnd[0]").sendVKey(0)
        self.session.findById("wnd[0]").sendVKey(11)
        self.back(2)

    def back(self, number_of_time = 1):
        for _ in range(0, number_of_time):
            self.session.findById("wnd[0]").sendVKey(3)
def read_config(config_path):
    with open(config_path, "r") as file:
        data = file.read()
    return literal_eval(data)

config_path = "config.txt"
config_data = read_config(config_path)
sap_id = config_data["sap_id"]
sap_pw = config_data["sap_pw"]
data_saving_path = config_data["data_file_path"]
sap_connection  = config_data["sap_connection"]
all_el_line = config_data["all_el_line"]
_current_el_line = config_data["current_el_line"] 
timedelay = config_data["timedelay"]


def submit_confirmation(event=None):
    present_time = datetime.now()
    if validation_date.year <= present_time.year and validation_date.month <= present_time.month and validation_date.day <= present_time.day:
        print("Pass")
    
    else:
        showerror("Error Code 100, Bazinga ")
        

    
root = Tk()
icon_img = r"img/logo.ico"
root.title("10-Stage Confirmation")
root.iconbitmap(icon_img)

frame_input = Frame(root)
frame_input.grid(row=0, column=0)

frame_datashow = Frame(root)
frame_datashow.grid(row=1, column=0)

module_input_label = Label(frame_input, text="Scan Module ID:", borderwidth=2, relief="groove")
module_input_label.grid(row=0, column=0)


module_input_entry = Entry(frame_input)
module_input_entry.grid(row=0, column=1)

module_submit_btn = Button(frame_input, text = "Submit", command = submit_confirmation)
module_submit_btn.grid(row=0, column=2)

module_input_entry.bind('<Return>', submit_confirmation)

root.mainloop()

True